Global setup



In [32]:
n=1
k=4
q=41 #Alkaline D
#q should be divisible by 8 for NTT
eta=1 #secret key range, prefered 2 in Alkaline
gamma1=2^4 #y coeff range, power of 2, exponent about 2/3 of q?  Earlier version was *not* power of 2
gamma2=(q-1)/4 #low-order rounding range, large(?) factor of q-1

In [33]:
tau = 1 #number of 1's in c
ln(binomial(4, tau)).n()+tau

2.38629436111989

In [34]:
beta = tau*eta
beta

1

Chance of $\mathbf{y}$-coefficient passing:

In [35]:
((2*(gamma1-beta)-1)/(2*gamma1-1))^(n*k).n()

0.765852644390328

Chance of low-order rounding passing:

In [36]:
((2*(gamma2-beta)-1)/(2*gamma2-1))^(n*k).n()

0.640886733527214

Expected number of repetitions:

In [37]:
1/(((2*(gamma1-beta)-1)/(2*gamma1-1))^(n*k))*1/(((2*(gamma2-beta)-1)/(2*gamma2-1))^(n*k)).n()

2.03738686545784

Alice generates a signing key

In [38]:
#A = random_matrix(ZZ, 4, 4, x=0, y=q)
A = matrix(4,4, 
[[ 4, 14,  4, 14],
[27, 28,  8, 40],
[ 9,  5,  8, 19],
[ 8,  9, 14,  3]])
A

[ 4 14  4 14]
[27 28  8 40]
[ 9  5  8 19]
[ 8  9 14  3]

In [39]:
#S1 = random_matrix(ZZ, 4, 4, x=-eta, y=eta+1)
S1 = matrix(ZZ, 
[[-1,  0, -1,  1],
[ 0,  1,  0,  1],
[ 1, -1,  0,  1],
[-1,  0,  0,  0]])
S1

[-1  0 -1  1]
[ 0  1  0  1]
[ 1 -1  0  1]
[-1  0  0  0]

In [40]:
#S2 = random_matrix(ZZ, 4, 4, x=-eta, y=eta+1)
S2 = matrix(ZZ, 
[[ 0, -1, -1,  0],
[ 1, -1,  0, -1],
[ 0, -1,  1,  1],
[ 0,  1,  0, -1]])
S2

[ 0 -1 -1  0]
[ 1 -1  0 -1]
[ 0 -1  1  1]
[ 0  1  0 -1]

$S_1, S_2$ is the private key


In [41]:
T = matrix(((A*S1 + S2) % q))
T

[27  9 36 22]
[24 19 14 21]
[21 37 33 23]
[ 3 37 33 30]

$(A,T)$ is the public key


Alice signs the message



In [42]:
M = 'hi!'
M

'hi!'

In [43]:
#y = random_matrix(ZZ, 4, 1, x=-(gamma1-1), y=gamma1)
y = matrix(4, 1, [-3, 0, -1, 14])
y

[-3]
[ 0]
[-1]
[14]

In [44]:
def high_bits(r, alpha):
    r = r % q
    r0 = Mod(r, alpha).lift_centered()
    if r-r0 == q-1:
        return 0
    else:
        return (r-r0)/alpha

def low_bits(r, alpha):
    r = r % q
    r0 = Mod(r, alpha).lift_centered()
    if r-r0 == q-1:
        return r0-1
    else:
        return r0

In [45]:
matrix(((A*y) % q))

[16]
[20]
[26]
[ 4]

In [46]:
w1 = matrix(((A*y) % q)).apply_map(lambda r: high_bits(r, 2*gamma2) )
w1

[1]
[1]
[1]
[0]

In [47]:
from sage.cpython.string import str_to_bytes

str_to_bytes(M+str(w1))

b'hi![1]\n[1]\n[1]\n[0]'

In [48]:
import hashlib

c_hash = hashlib.shake_256(str_to_bytes(M+str(w1)))

In [49]:
n*k

4

In [50]:
def sample_in_ball(digest):  # digest should be twice the length of c
    c = [0] * (n*k)
    for i in range(n*k-tau, n*k):
        j = digest[i] % (i+1)
        s = digest[i+n*k] % 2
        c[i] = c[j]
        c[j] = (-1)^s
    return c
    

In [51]:
c = matrix(4, 1, sample_in_ball(c_hash.digest(2*n*k)))
c

[1]
[0]
[0]
[0]

In [52]:
z = matrix(((y + S1*c) % q))
z.mod(q).lift_centered()

[-4]
[ 0]
[ 0]
[13]

Is $\|\mathbf{z}\|_\infty \geq \gamma_1-\beta$?

In [53]:
gamma1-beta

15

If so, reject and pick a new $\mathbf{y}$

Is $\|\mathrm{LowBits}(\mathbf{Ay}-c\mathbf{s}_2, 2\gamma_2)\|_\infty \geq \gamma_2-\beta$?

In [54]:
matrix(((A*y - S2*c) % q)).apply_map(lambda r: low_bits(r, 2*gamma2) )

[-4]
[-1]
[ 6]
[ 4]

In [55]:
gamma2-beta

9

If so, reject and pick a new $\mathbf{y}$

If not, $(\mathbf{z}, \mathbf{c})$ is the signature


In [56]:
(z,c)

(
[37]  [1]
[ 0]  [0]
[ 0]  [0]
[13], [0]
)

Bob verifies the signature

In [57]:
w1prime = matrix(((A*z-T*c) % q)).apply_map(lambda r: high_bits(r, 2*gamma2) )
w1prime

[1]
[1]
[1]
[0]

Is $\|\mathbf{z}\|_\infty < \gamma_1-\beta$?

In [58]:
z.mod(q).lift_centered()

[-4]
[ 0]
[ 0]
[13]

In [59]:
gamma1-beta

15

Yes

Is $c = H(M \| w_1')$?

In [60]:
cprime_hash = hashlib.shake_256(str_to_bytes(M+str(w1prime)))
cprime = matrix(4, 1, sample_in_ball(cprime_hash.digest(8)))
cprime

[1]
[0]
[0]
[0]

In [61]:
c == cprime

True

Yes.  The signature is accepted.